This sequence-to-sequence experiment aims to learn joint representation of document in one particular language (e.g. English) to document in another language (e.g. Dutch). For this first stage experiment, we use a parallel corpus from europarl (the one available in data/europarl_*_test is a splitted sample of first paragraph from the original europarl text document - for a purpose of tutorial).

In [1]:
from text_preprocessing import *

In [3]:
from language_models import *